In [2]:
from sympy import symbols, Function, Eq, dsolve, sin

# 定义变量和函数
x = symbols('x')
y = Function('y')(x)
z = Function('z')(x)

# 创建微分方程组
eq1 = Eq(y.diff(x) + 2*y - z, 0)
eq2 = Eq(z.diff(x) - y + 2*z, 0)

# 使用dsolve函数求解微分方程组
solutions = dsolve((eq1, eq2))

# 打印解
for solution in solutions:
    print(solution)

Eq(y(x), -C1*exp(-3*x) + C2*exp(-x))
Eq(z(x), C1*exp(-3*x) + C2*exp(-x))


In [2]:
from sympy import symbols, Function, Eq, dsolve, sin

omega2, omega1, t, l = symbols('omega2 omega1 t lmabda')

theta2 = Function('theta2')(t)

eq1 = Eq(theta2.diff(t), sin(omega1*t - theta2))

solutions = dsolve((eq1))

solutions
# for solution in solutions:
#     print(solution)

KeyboardInterrupt: 